In [1]:
import pandas as pd
from sklearn import decomposition, preprocessing, cluster, tree
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

In [2]:
#create dataframes from csv files

peptides = pd.read_csv("PD-datasets/train_peptides.csv")
proteins = pd.read_csv("PD-datasets/train_proteins.csv")
clinical = pd.read_csv("PD-datasets/train_clinical_data.csv")

In [3]:
pep_pivot = peptides.pivot(index="visit_id",columns="Peptide",values="PeptideAbundance").sort_values(by="visit_id")

In [4]:
pro_pivot = proteins.pivot(index="visit_id",columns="UniProt",values="NPX")

In [5]:
merged_pivots = pep_pivot.merge(pro_pivot,how='left',on='visit_id')
print(merged_pivots.shape)
merged_pivots.head()

(1113, 1195)


,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,ADDKETC(UniMod_4)FAEEGK,ADDKETC(UniMod_4)FAEEGKK,ADDLGKGGNEESTKTGNAGSR,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,6580710.0,31204.4,7735070.0,NaN,NaN,NaN,46620.3,236144.0,NaN,NaN,...,NaN,9469.45,94237.6,NaN,23016.0,177983.0,65900.0,15382.0,NaN,19017.40
10053_12,6333510.0,52277.6,5394390.0,NaN,NaN,NaN,57554.5,108298.0,45885.4,NaN,...,NaN,14408.40,NaN,NaN,28537.0,171733.0,65668.1,NaN,9295.65,25697.80
10053_18,7129640.0,61522.0,7011920.0,35984.7,17188.0,19787.3,36029.4,708729.0,5067790.0,30838.2,...,317477.0,38667.20,111107.0,NaN,37932.6,245188.0,59986.1,10813.3,NaN,29102.70
10138_12,7404780.0,46107.2,10610900.0,NaN,20910.2,66662.3,55253.9,79575.5,6201210.0,26720.0,...,557904.0,44556.90,155619.0,14647.9,36927.7,229232.0,106564.0,26077.7,21441.80,7642.42
10138_24,13788300.0,56910.3,6906160.0,13785.5,11004.2,63672.7,36819.8,34160.9,2117430.0,15645.2,...,NaN,47836.70,177619.0,17061.1,25510.4,176722.0,59471.4,12639.2,15091.40,6168.55


In [6]:
full_merge = merged_pivots.merge(clinical,how='left',on='visit_id')
print(full_merge.shape)
full_merge.head()

(1113, 1203)


,visit_id,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,ADDKETC(UniMod_4)FAEEGK,ADDKETC(UniMod_4)FAEEGKK,...,Q9UNU6,Q9Y646,Q9Y6R7,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,10053_0,6580710.0,31204.4,7735070.0,NaN,NaN,NaN,46620.3,236144.0,NaN,...,15382.0,NaN,19017.40,10053.0,0.0,3.0,0.0,13.0,0.0,NaN
1,10053_12,6333510.0,52277.6,5394390.0,NaN,NaN,NaN,57554.5,108298.0,45885.4,...,NaN,9295.65,25697.80,10053.0,12.0,4.0,2.0,8.0,0.0,NaN
2,10053_18,7129640.0,61522.0,7011920.0,35984.7,17188.0,19787.3,36029.4,708729.0,5067790.0,...,10813.3,NaN,29102.70,10053.0,18.0,2.0,2.0,0.0,0.0,NaN
3,10138_12,7404780.0,46107.2,10610900.0,NaN,20910.2,66662.3,55253.9,79575.5,6201210.0,...,26077.7,21441.80,7642.42,10138.0,12.0,3.0,6.0,31.0,0.0,On
4,10138_24,13788300.0,56910.3,6906160.0,13785.5,11004.2,63672.7,36819.8,34160.9,2117430.0,...,12639.2,15091.40,6168.55,10138.0,24.0,4.0,7.0,19.0,10.0,On


In [7]:
le = LabelEncoder()
encoded_mm = mega_merge[["upd23b_clinical_state_on_medication"]].apply(le.fit_transform)
full_merge["upd23b_clinical_state_on_medication"] = encoded_mm["upd23b_clinical_state_on_medication"]
mega_merge.head()

NameError: name 'mega_merge' is not defined